In [ ]:
# WORKING TEST

def test(model, nodes, val_edges, z, k=50):
    model.eval()  # Set the model to evaluation mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # add this line to set the device

    # Take 5 samples from val_edges as positive examples


    with torch.no_grad():

        # Convert V to a boolean tensor for faster lookup.
        v_mask = torch.zeros(num_nodes, dtype=torch.bool)
        v_mask[nodes] = True
        v_mask = v_mask.to(device)

        # Assume val_edges contains the validation edges (it should be a 2 x num_val_edges tensor)
        # val_edges = ...

        # Check if both nodes of each edge in val_edges are in V
        source_nodes = val_edges[0, :]
        target_nodes = val_edges[1, :]
        can_exist_in_V = v_mask[source_nodes] & v_mask[target_nodes]

        # Filter the edges that can exist in V
        valid_edges_in_V = val_edges[:, can_exist_in_V]
        positive_pairs = valid_edges_in_V


        # FOR MEMORY
        selected_pairs = positive_pairs[:, torch.randint(valid_edges_in_V.size(1), (1,))]


        # --- Generating negative pairs ---

        # Find the unique starting nodes in val_edges
        start_nodes = torch.unique(selected_pairs[0, :]).to(device)

        # Tour over start nodes

        print(start_nodes)
        print("+++++++++")
        # Generate all possible pairs from start_nodes to all nodes in V
        all_possible_pairs = torch.stack(torch.meshgrid(start_nodes, V), dim=-1).reshape(-1, 2).t().to(device)


        # Remove the existing edges in val_edges from all_possible_pairs to create the negative pairs
        existing_pairs = positive_pairs.t()
        existing_pairs = existing_pairs.to(device)

        # Removing positive pairs that are generated accidentaly
        negative_pairs = remove_common_edges(E_all=positive_pairs,B=all_possible_pairs) # B - (A INTERSECTION B)

        # Negative examples for validation
        positive_scores = model.decode(z, positive_pairs)
        negative_scores = model.decode(z, negative_pairs)

        # Combine positive edges and negative scores
        all_edges = torch.cat([positive_pairs, negative_pairs], dim=1)
        all_scores = torch.cat([positive_scores, negative_scores])
        # Indicate which edges are positive (1 for positive, 0 for negative)
        positive_edge_indicator = torch.tensor([1]*positive_pairs.size(1) + [0]*negative_pairs.size(1)).to(device)


        recall= calculate_recall_per_node(all_edges, all_scores, positive_edge_indicator, k,start_nodes)


        return recall


In [ ]:
import torch

def calculate_recall_per_node(all_edges, all_scores, positive_edge_indicator, K, start_nodes):
    """
    Calculate recall for each individual starting node using tensor operations on GPU.

    Parameters:
    - all_edges: Tensor of shape [2, num_edges], containing edges (source -> target).
    - all_scores: Tensor of shape [num_edges], containing scores for each edge.
    - positive_edge_indicator: Tensor of shape [num_edges], containing 1 for positive edges and 0 for negative edges.
    - K: The number of top edges to consider for calculating recall.

    Returns:
    - recall_per_node: Dictionary with nodes as keys and recall as values.
    """

    print("all_edges")
    print(all_edges.shape)

    print("all_scores")
    print(all_scores.shape)

    print("positive_edge_indicator")
    print(positive_edge_indicator.shape)





    # Sort scores in descending order
    sorted_indices = torch.argsort(all_scores, descending=True)

    all_edges = all_edges[:, sorted_indices]
    positive_edge_indicator = positive_edge_indicator[sorted_indices]


    recall = 0
    for start_node in start_nodes:
      # Find all edges related to start_node
      mask = all_edges[0,:] == start_node

      filtered_indicators = positive_edge_indicator[mask]

      positive_edge_indicator = torch.masked_select(positive_edge_indicator, torch.logical_not(mask))

      recall = recall + filtered_indicators[:K].sum() / filtered_indicators.sum()
      print(filtered_indicators[:K].sum())
      print(filtered_indicators.sum())

    recall = recall / len(start_nodes)



    # Create bins for each unique start node
    #bins = torch.zeros_like(all_scores).scatter_(0, all_edges[0, :], 1).cumsum(0)

    # Create a mask for top K elements in each bin
    #top_k_mask = (bins <= K).gather(0, torch.argsort(bins.gather(0, sorted_indices)))

    # Compute recalls by start node
    #top_k_sorted_positive_indicators = positive_edge_indicator[sorted_indices][top_k_mask]
    #recall = (top_k_sorted_positive_indicators.view(len(start_nodes), -1).sum(1) / K).cpu().numpy()

    # Create recall_per_node dictionary
    #recall_per_node = {node.item(): recall for node, recall in zip(start_nodes, recalls)}

    return recall

# Example usage remains the same
